# Homework: Agents

In this homework, we will learn more about function calling, and we will also explore MCP - model-context protocol.

## Preparation

First, we'll define the functions that we will use when building our agent. They will generate and set fake weather data.

In [2]:
import random

# Pre-defined weather data for a specific city
known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    """Retrieves the temperature for a specified city."""
    # Clean and standardize the city name
    city = city.strip().lower()

    # Check if the city is in our known data
    if city in known_weather_data:
        return known_weather_data[city]

    # If not, generate a random temperature
    return round(random.uniform(-5, 35), 1)


def set_weather(city: str, temp: float) -> str:
    """Sets the temperature for a specified city."""
    # Clean and standardize the city name
    city = city.strip().lower()
    # Update the known weather data with the new temperature
    known_weather_data[city] = temp
    # Return a confirmation message
    return 'OK'

## Q1. Define function description

We want to use `get_weather` as a tool for our agent, so we need to describe it in a format the LLM can understand. This involves defining its name, purpose, parameters, and which parameters are required.

- **TODO1 (name):** The name of the function is `get_weather`.
- **TODO2 (description):** A clear, concise description of what the function does. "Get the current temperature for a given city." is a good choice.
- **TODO3 (parameter name):** The function takes one parameter, which is `city`.
- **TODO4 (parameter description):** A description of the parameter. "The city for which to get the weather" clearly explains its purpose.
- **TODO5 (required):** The `city` parameter is essential for the function to work, so it must be included in the `required` list.

In [3]:
get_weather_tool = {
    "type": "function",
    # The 'function' key is standard for OpenAI tool definitions.
    "function": {
        "name": "get_weather",
        "description": "Get the current temperature for a given city.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city for which to get the weather, e.g. Berlin"
                }
            },
            "required": ["city"],
        }
    }
}

print(f"The value for TODO3 is: '{list(get_weather_tool['function']['parameters']['properties'].keys())[0]}'" )

The value for TODO3 is: 'city'


### Q1 (Optional) Testing the Tool

To test if our tool definition works, we can make a call to an LLM (like OpenAI's GPT models) and see if it correctly identifies that it needs to use our `get_weather` tool based on a user's prompt. We'll simulate a conversation where the user asks for the weather.

In [4]:
import os
from openai import OpenAI
import json

# To run this, you need an OpenAI API key.
# You can set it as an environment variable 'OPENAI_API_KEY'
# client = OpenAI()

# Since we don't want to make a real API call, we'll mock the client and response.
# This is how a real response would look.
mock_response = {
    'choices': [
        {
            'message': {
                'role': 'assistant',
                'tool_calls': [
                    {
                        'id': 'call_abc123',
                        'type': 'function',
                        'function': {
                            'name': 'get_weather',
                            'arguments': '{\n  "city": "Berlin"\n}'
                        }
                    }
                ]
            }
        }
    ]
}

print("Simulating an API call to test the tool definition.")
print("User prompt: What's the weather like in Berlin?")
print("--- Mock LLM Response ---")

# The response contains a 'tool_calls' object, indicating the LLM wants to use our function.
tool_call = mock_response['choices'][0]['message']['tool_calls'][0]
function_name = tool_call['function']['name']
function_args = json.loads(tool_call['function']['arguments'])

print(f"LLM wants to call function: {function_name}")
print(f"With arguments: {function_args}")

# Now we would call our actual Python function with these arguments
if function_name == 'get_weather':
    weather_result = get_weather(city=function_args.get('city'))
    print(f"\nCalling our local get_weather() function...\nResult: {weather_result}")

Simulating an API call to test the tool definition.
User prompt: What's the weather like in Berlin?
--- Mock LLM Response ---
LLM wants to call function: get_weather
With arguments: {'city': 'Berlin'}

Calling our local get_weather() function...
Result: 20.0


## Q2. Adding another tool

Next, we'll write a description for the `set_weather` function. This function takes two parameters: `city` (a string) and `temp` (a float). Both are required.

The description should clearly state that this function sets or updates the weather data. The parameters section must define both `city` and `temp`, specifying their types (`string` and `number`) and what they represent. Finally, the `required` array must list both `'city'` and `'temp'`.

In [5]:
import json

set_weather_tool = {
    "type": "function",
    "function": {
        "name": "set_weather",
        "description": "Set or update the temperature for a specified city in the database.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The name of the city, e.g. London"
                },
                "temp": {
                    "type": "number",
                    "description": "The temperature to set for the city."
                }
            },
            "required": ["city", "temp"]
        }
    }
}

print("Description for set_weather function:")
print(json.dumps(set_weather_tool, indent=2))

Description for set_weather function:
{
  "type": "function",
  "function": {
    "name": "set_weather",
    "description": "Set or update the temperature for a specified city in the database.",
    "parameters": {
      "type": "object",
      "properties": {
        "city": {
          "type": "string",
          "description": "The name of the city, e.g. London"
        },
        "temp": {
          "type": "number",
          "description": "The temperature to set for the city."
        }
      },
      "required": [
        "city",
        "temp"
      ]
    }
  }
}


## Q3. Install FastMCP


In [6]:
!pip show fastmcp

Name: fastmcp
Version: 2.10.5
Summary: The fast, Pythonic way to build MCP servers and clients.
Home-page: https://gofastmcp.com
Author: Jeremiah Lowin
Author-email: 
License-Expression: Apache-2.0
Location: /home/dan/environments/.env_dataeng/lib/python3.10/site-packages
Requires: authlib, cyclopts, exceptiongroup, httpx, mcp, openapi-pydantic, pydantic, pyperclip, python-dotenv, rich
Required-by: 


## Q4. Simple MCP Server

We will create a simple MCP server using our weather functions. First, let's create the `weather_server.py` file. We need to add proper docstrings to our functions so `FastMCP` can automatically generate the tool descriptions.

When we run this server script from the command line, it will print a startup message. This message indicates how the server is communicating. The part we are looking for is the transport method.

```
Starting MCP server 'Weather Service 🌦️' with transport 'stdio'
```

The value instead of `<TODO>` is **stdio**.

In [7]:
%%writefile weather_server.py
import random
from fastmcp import FastMCP

# Initialize the MCP server with a name
mcp = FastMCP("Weather Service 🌦️")

# Pre-defined weather data
known_weather_data = {
    'berlin': 20.0
}

# The @mcp.tool decorator registers the function as an available tool
@mcp.tool
def get_weather(city: str) -> float:
    """
    Retrieves the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to retrieve weather data.

    Returns:
        float: The temperature associated with the city.
    """
    city = city.strip().lower()
    if city in known_weather_data:
        return known_weather_data[city]
    return round(random.uniform(-5, 35), 1)

@mcp.tool
def set_weather(city: str, temp: float) -> str:
    """
    Sets the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to set the weather data.
        temp (float): The temperature to associate with the city.

    Returns:
        str: A confirmation string 'OK' indicating successful update.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

# This block ensures the server runs only when the script is executed directly
if __name__ == "__main__":
    mcp.run()

Overwriting weather_server.py


In [8]:
## Q5. Protocol
# Simulate calling the tool manually
import json

rpc_request = {
    "jsonrpc": "2.0",
    "id": 3,
    "method": "tools/call",
    "params": {
        "name": "get_weather",
        "arguments": {
            "city": "Berlin"
        }
    }
}

print("Request:")
print(json.dumps(rpc_request, indent=2))

# Simulate handling
response = get_weather(**rpc_request["params"]["arguments"])
rpc_response = {
    "jsonrpc": "2.0",
    "id": rpc_request["id"],
    "result": response
}

print("\nResponse:")
print(json.dumps(rpc_response, indent=2))

Request:
{
  "jsonrpc": "2.0",
  "id": 3,
  "method": "tools/call",
  "params": {
    "name": "get_weather",
    "arguments": {
      "city": "Berlin"
    }
  }
}

Response:
{
  "jsonrpc": "2.0",
  "id": 3,
  "result": 20.0
}


## Q6. Client



In [9]:
import asyncio
import json
from fastmcp import Client
import weather_server # Import the server module when in a notebook
import nest_asyncio

# Apply the patch to allow nested event loops in Jupyter
nest_asyncio.apply()

async def main():
    """
    An async function that connects to the MCP server and lists the available tools
    using the correct `list_tools()` method.
    """
    print("Attempting to connect to the MCP server...")
    async with Client(weather_server.mcp) as mcp_client:
        print("Connection successful. Listing tools...")
        # The correct method to get the list of tool objects is `list_tools()`.
        tools = await mcp_client.list_tools()
    return tools

# Now we can run the async main function
print("--- Running Q6 Client Test ---")
available_tools = asyncio.run(main())

# Print the result if found
if available_tools:
    print("\n--- Available Tools Found ---")
    
    # --- JSON Serialization Fix ---
    # The result `available_tools` is a list of custom `Tool` objects.
    # The `json.dumps` function throws a TypeError because it doesn't know
    # how to convert these custom objects into a JSON string.
    # We can solve this by converting the list of objects into a list of dictionaries
    # using a list comprehension and Python's built-in `vars()` function.
    print("Converting Tool objects to a JSON-serializable format...")
    serializable_tools = [vars(tool) for tool in available_tools]

    # Now, `json.dumps` will work correctly with the list of dictionaries.
    print(json.dumps(serializable_tools, indent=2))

else:
    print("\nCould not retrieve the list of tools.")

--- Running Q6 Client Test ---
Attempting to connect to the MCP server...
Connection successful. Listing tools...

--- Available Tools Found ---
Converting Tool objects to a JSON-serializable format...
[
  {
    "name": "get_weather",
    "title": null,
    "description": "Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.",
    "inputSchema": {
      "properties": {
        "city": {
          "title": "City",
          "type": "string"
        }
      },
      "required": [
        "city"
      ],
      "type": "object"
    },
    "outputSchema": {
      "properties": {
        "result": {
          "title": "Result",
          "type": "number"
        }
      },
      "required": [
        "result"
      ],
      "title": "_WrappedResult",
      "type": "object",
      "x-fastmcp-wrap-result": true
    },
    "annotations": null,
   

## Q6: List tools and call get_weather using FastMCP Client in Jupyter

In [11]:
# Q6: List tools and call get_weather using FastMCP Client in Jupyter
import asyncio
from fastmcp import Client
import weather_server
import nest_asyncio
nest_asyncio.apply()

async def main():
    async with Client(weather_server.mcp) as mcp_client:
        print("--- Available Tools (Q6) ---")
        tools = await mcp_client.list_tools()
        for tool in tools:
            print(tool)
        print("\n--- Call get_weather (Q5) ---")
        result = await mcp_client.call_tool("get_weather", {"city": "Berlin"})
        print("Result of get_weather for Berlin:", result)

await main()

--- Available Tools (Q6) ---
name='get_weather' title=None description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.' inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'} outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True} annotations=None meta=None
name='set_weather' title=None description="Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update." inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'temp': {'titl

In [12]:
#with the code and client provided in the homework
import mcp_client

# Start the MCP server and client
our_mcp_client = mcp_client.MCPClient(["python", "weather_server.py"])
our_mcp_client.start_server()
our_mcp_client.initialize()
our_mcp_client.initialized()

# Q6: List available tools
tools = our_mcp_client.get_tools()
print("--- Available Tools (Q6) ---")
print(tools)

# Q5: Call get_weather for Berlin
result = our_mcp_client.call_tool('get_weather', {'city': 'Berlin'})
print("\n--- Call get_weather (Q5) ---")
print(result)

Started server with command: python weather_server.py
Sending initialize request...
Initialize response: {'protocolVersion': '2024-11-05', 'capabilities': {'experimental': {}, 'prompts': {'listChanged': False}, 'resources': {'subscribe': False, 'listChanged': False}, 'tools': {'listChanged': True}}, 'serverInfo': {'name': 'Weather Service 🌦️', 'version': '1.11.0'}}
Sending initialized notification...
Handshake completed successfully
Retrieving available tools...
Available tools: ['get_weather', 'set_weather']
--- Available Tools (Q6) ---
[{'name': 'get_weather', 'description': 'Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.', 'inputSchema': {'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, 'outputSchema': {'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required'

In [13]:
import json

class MCPTools:
    def __init__(self, mcp_client):
        self.mcp_client = mcp_client
        self.tools = None
    
    def get_tools(self):
        if self.tools is None:
            mcp_tools = self.mcp_client.get_tools()
            self.tools = convert_tools_list(mcp_tools)
        return self.tools

    def function_call(self, tool_call_response):
        function_name = tool_call_response.name
        arguments = json.loads(tool_call_response.arguments)

        result = self.mcp_client.call_tool(function_name, arguments)

        return {
            "type": "function_call_output",
            "call_id": tool_call_response.call_id,
            "output": json.dumps(result, indent=2),
        }

In [20]:
from dotenv import load_dotenv
import os
from dotenv import load_dotenv

# This line loads the environment variables from the .env file
load_dotenv()

# Now you can access the API key using os.getenv()
api_key = os.getenv("OPENAI_API_KEY")

# It's good practice to check if the key was loaded successfully
if not api_key:
    raise ValueError("API_KEY not found. Make sure it's set in your .env file.")

# You can now use the api_key variable to connect to your service
print("API Key loaded successfully!")
# Example: client = YourApiService(api_key=api_key)

API Key loaded successfully!


In [21]:
from openai import OpenAI
client = OpenAI()

In [22]:
# model = "gpt-4.1-nano-2025-04-14"
# def llm(prompt):
#     response = client.chat.completions.create(
#         model= model, #'gpt-4o-mini',
#         messages=[{"role": "user", "content": prompt}]
#     )
    
#     return response.choices[0].message.content

In [23]:
import chat_assistant

our_mcp_client = mcp_client.MCPClient(["python", "weather_server.py"])

our_mcp_client.start_server()
our_mcp_client.initialize()
our_mcp_client.initialized()

mcp_tools = mcp_client.MCPTools(mcp_client=our_mcp_client)


developer_prompt = """
You help users find out the weather in their cities. 
If they didn't specify a city, ask them. Make sure we always use a city.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=mcp_tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

chat.run()

Started server with command: python weather_server.py
Sending initialize request...
Initialize response: {'protocolVersion': '2024-11-05', 'capabilities': {'experimental': {}, 'prompts': {'listChanged': False}, 'resources': {'subscribe': False, 'listChanged': False}, 'tools': {'listChanged': True}}, 'serverInfo': {'name': 'Weather Service 🌦️', 'version': '1.11.0'}}
Sending initialized notification...
Handshake completed successfully


You: what's the weather like in Berlin?


Retrieving available tools...
Available tools: ['get_weather', 'set_weather']
Calling tool 'get_weather' with arguments: {'city': 'Berlin'}


You: exit


KeyboardInterrupt: Interrupted by user

You: ff
